# Trying For Question Answering model 
* Subject- Theory of Computation

In [24]:
import faiss
import numpy as np
from transformers import pipeline
from langchain_community.document_loaders import WikipediaLoader
from bs4 import BeautifulSoup
import requests
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import CharacterTextSplitter

# Load Documents from Multiple Sources:

In [14]:
# Load documents from Wikipedia
wikipedia_loader = WikipediaLoader(query="Theory of Computation")
wikipedia_documents = wikipedia_loader.load()

# Load documents from Javatpoint
javatpoint_url = "https://www.javatpoint.com/automata-tutorial"
javatpoint_response = requests.get(javatpoint_url)
javatpoint_soup = BeautifulSoup(javatpoint_response.content, features="lxml")
javatpoint_content = javatpoint_soup.get_text()

# Load documents from GeeksforGeeks
geeksforgeeks_url = "https://www.geeksforgeeks.org/introduction-of-theory-of-computation/"
geeksforgeeks_response = requests.get(geeksforgeeks_url)
geeksforgeeks_soup = BeautifulSoup(geeksforgeeks_response.content, features="lxml")
geeksforgeeks_content = geeksforgeeks_soup.get_text()

# Load documents from Geektonight
geektonight_url = "https://www.geektonight.com/theory-of-computation-notes/"
geektonight_response = requests.get(geektonight_url)
geektonight_soup = BeautifulSoup(geektonight_response.content, features="lxml")
geektonight_content = geektonight_soup.get_text()

# Combine all content to create full context 

In [34]:
# Combine all document contents
combined_docs = [
    doc.page_content for doc in wikipedia_documents
] + [javatpoint_content, geeksforgeeks_content, geektonight_content]
combined_docs

['In theoretical computer science and mathematics, the theory of computation is the branch that deals with what problems can be solved on a model of computation, using an algorithm, how efficiently they can be solved or to what degree (e.g., approximate solutions versus precise ones).  The field is divided into three major branches: automata theory and formal languages, computability theory, and computational complexity theory, which are linked by the question: "What are the fundamental capabilities and limitations of computers?".\nIn order to perform a rigorous study of computation, computer scientists work with a mathematical abstraction of computers called a model of computation. There are several models in use, but the most commonly examined is the Turing machine. Computer scientists study the Turing machine because it is simple to formulate, can be analyzed and used to prove results, and because it represents what many consider the most powerful possible "reasonable" model of comp

In [53]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=10000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

# Split the document into chunks:

In [54]:
# Combine all documents into a single text and split by \n\n
combined_text = "\n\n".join(combined_docs)
chunks = text_splitter.split_text(combined_text)
chunks

['In theoretical computer science and mathematics, the theory of computation is the branch that deals with what problems can be solved on a model of computation, using an algorithm, how efficiently they can be solved or to what degree (e.g., approximate solutions versus precise ones).  The field is divided into three major branches: automata theory and formal languages, computability theory, and computational complexity theory, which are linked by the question: "What are the fundamental capabilities and limitations of computers?".\nIn order to perform a rigorous study of computation, computer scientists work with a mathematical abstraction of computers called a model of computation. There are several models in use, but the most commonly examined is the Turing machine. Computer scientists study the Turing machine because it is simple to formulate, can be analyzed and used to prove results, and because it represents what many consider the most powerful possible "reasonable" model of comp

# Generate Embedding and Create FIASS index

In [40]:
# Build the Faiss index
dimension = embeddings_array.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_array)

In [41]:
# Define the QA pipeline with FLAN-T5 model
qa_pipeline = pipeline("question-answering", model="google/flan-t5-base")

Some weights of T5ForQuestionAnswering were not initialized from the model checkpoint at google/flan-t5-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Set Up the QA Pipeline:

In [44]:
# Define the QA pipeline with FLAN-T5 model
# Use a local transformer model for question answering
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")
qa_pipeline

# Getting answer 

In [42]:
# Function to retrieve relevant context and get the answer
def query_retriever(query, k=5):
    # Generate the embedding for the query
    query_embedding = model.encode(query).reshape(1, -1)

    # Perform the search on the FAISS index
    distances, indices = index.search(query_embedding, k)

    # Retrieve the most relevant chunks
    relevant_chunks = [chunks[idx]['page_content'] for idx in indices[0]]
    relevant_context = " ".join(relevant_chunks)

    # Get the answer with dynamically refined context
    answer = qa_pipeline(question=query, context=relevant_context)
    return answer['answer']

In [43]:
# Get the user's question
question = input("Enter your question: ")

# Get and print the answer
answer = query_retriever(question, k=5)
print(answer)

 Kleene Star Closure in TOC?What


# Using another model 